In [17]:
# -*- coding: utf-8 -*-

import os
import random
import string
import oss2


# 以下代码展示了文件上传的高级用法，如断点续传、分片上传等。
# 基本的文件上传如上传普通文件、追加文件，请参见object_basic.py


# 首先初始化AccessKeyId、AccessKeySecret、Endpoint等信息。
# 通过环境变量获取，或者把诸如“<你的AccessKeyId>”替换成真实的AccessKeyId等。
#
# 以杭州区域为例，Endpoint可以是：
#   http://oss-cn-hangzhou.aliyuncs.com
#   https://oss-cn-hangzhou.aliyuncs.com
# 分别以HTTP、HTTPS协议访问。
access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', 'LTAIvAK9dpqW7jlT')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '31ilGy5Z8Pvnv3GsZRvSWDEa1mFtog')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'bayesba-test1')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'http://oss-cn-beijing.aliyuncs.com')


# 确认上面的参数都填写正确了
for param in (access_key_id, access_key_secret, bucket_name, endpoint):
    print(param)
    assert '<' not in param, '请设置参数：' + param

# 创建Bucket对象，所有Object相关的接口都可以通过Bucket对象来进行
bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)


def random_string(n):
    return ''.join(random.choice(string.ascii_lowercase) for i in range(n))

# 生成一个本地文件用于测试。文件内容是bytes类型。
filename = random_string(32) + '.txt'
content = oss2.to_bytes(random_string(1024 * 1024))

with open(filename, 'wb') as fileobj:
    fileobj.write(content)


"""
断点续传上传
"""

# 断点续传一：因为文件比较小（小于oss2.defaults.multipart_threshold），
# 所以实际上用的是oss2.Bucket.put_object
oss2.resumable_upload(bucket, 'mongo_bak/remote-normal.txt', filename)

# 断点续传二：为了展示的需要，我们指定multipart_threshold可选参数，确保使用分片上传
oss2.resumable_upload(bucket, 'mongo_bak/remote-multipart.txt', filename, multipart_threshold=100 * 1024)


"""
分片上传
"""

# # 也可以直接调用分片上传接口。
# # 首先可以用帮助函数设定分片大小，设我们期望的分片大小为128KB
# total_size = os.path.getsize(filename)
# part_size = oss2.determine_part_size(total_size, preferred_size=128 * 1024)

# # 初始化分片上传，得到Upload ID。接下来的接口都要用到这个Upload ID。
# key = 'remote-multipart2.txt'
# upload_id = bucket.init_multipart_upload(key).upload_id

# # 逐个上传分片
# # 其中oss2.SizedFileAdapter()把fileobj转换为一个新的文件对象，新的文件对象可读的长度等于num_to_upload
# with open(filename, 'rb') as fileobj:
#     parts = []
#     part_number = 1
#     offset = 0
#     while offset < total_size:
#         num_to_upload = min(part_size, total_size - offset)
#         result = bucket.upload_part(key, upload_id, part_number,
#                                     oss2.SizedFileAdapter(fileobj, num_to_upload))
#         parts.append(oss2.models.PartInfo(part_number, result.etag))

#         offset += num_to_upload
#         part_number += 1

#     # 完成分片上传
#     bucket.complete_multipart_upload(key, upload_id, parts)

# # 验证一下
# with open(filename, 'rb') as fileobj:
#     assert bucket.get_object(key).read() == fileobj.read()

os.remove(filename)
print('end-end')

LTAIvAK9dpqW7jlT
31ilGy5Z8Pvnv3GsZRvSWDEa1mFtog
bayesba-test1
http://oss-cn-beijing.aliyuncs.com
end-end


In [3]:
# -*- coding: utf-8 -*-

import os
import oss2
help(os.getenv)
help(oss2.Auth)

Help on function getenv in module os:

getenv(key, default=None)
    Get an environment variable, return None if it doesn't exist.
    The optional second argument can specify an alternate default.
    key, default and the result are str.

Help on class Auth in module oss2.auth:

class Auth(builtins.object)
 |  用于保存用户AccessKeyId、AccessKeySecret，以及计算签名的对象。
 |  
 |  Methods defined here:
 |  
 |  __init__(self, access_key_id, access_key_secret)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [1]:
import happybase
import numpy as np
import pandas as pd
import pdhbase as pdh

def get_hbase_db():
    conn = happybase.Connection('47.93.144.37')
    return conn
conn = get_hbase_db()
tables = conn.tables()
print(tables)

# df = pdh.read_hbase(conn, 'test_1', 'row-key-1', 'cf1')
# print(df)
# families = {
#     'cf': dict(max_versions=10),
#     'cf1': dict(max_versions=1, block_cache_enabled=False),
#     'cf2': dict(),  # use defaults
# }
# conn.create_table(b'suncc_test', families)
# tbl1 = conn.table(b'suncc_test')
#单条存入
# tbl1.put('rowkey00000001',{'cf:name': 'karry', 'cf:age': '28', 'cf:addr': '山东'})
# tbl1.put('rowkey00000002',{'cf:name': 'alisa', 'cf:age': '3', 'cf:addr': '西安'})
# tbl1.put('rowkey00000003',{'cf:name': 'xiaoshan', 'cf:age': '25', 'cf:addr': '商洛'})
# tbl1.put('rowkey00000004',{'cf:name': 'zhaoyao', 'cf:age': '26', 'cf:addr': '周至'})
# tbl1.put('rowkey00000005',{'cf:name': 'baorui', 'cf:age': '38', 'cf:addr': '子长'})
# tbl1.put('rowkey00000006',{'cf:name': 'weiwei', 'cf:age': '34', 'cf:addr': '延安'})
# tbl1.put('rowkey00000007',{'cf:name': 'wangmeng', 'cf:age': 33, 'cf:addr': '延安'})
#遍历表
tbl = conn.table(b'suncc_test')
for key, data in tbl.scan():
    print(key, data)  
# row1 = tbl.row('rowkey00000001', columns=['cf'])
# print(row1)
# help(tbl.scan)

[b'cellphone_comment', b'cellphonestop_word', b'generalstop_word', b'interjection_word', b'lianci', b'member', b'phonepraise_in_comments', b'phonepraise_in_phoneinfo', b'split_symbol', b'suncc_test']
b'0000000000000001' {b'cf:addr': b'\xe5\xb1\xb1\xe4\xb8\x9c', b'cf:age': b'28', b'cf:name': b'karry'}
b'0000000000000002' {b'cf:addr': b'\xe8\xa5\xbf\xe5\xae\x89', b'cf:age': b'3', b'cf:name': b'alisa'}
b'0000000000000003' {b'cf:addr': b'\xe5\x95\x86\xe6\xb4\x9b', b'cf:age': b'25', b'cf:name': b'xiaoshan'}
b'0000000000000004' {b'cf:addr': b'\xe5\x91\xa8\xe8\x87\xb3', b'cf:age': b'26', b'cf:name': b'zhaoyao'}
b'0000000000000005' {b'cf:addr': b'\xe5\xad\x90\xe9\x95\xbf', b'cf:age': b'38', b'cf:name': b'baorui'}
b'0000000000000006' {b'cf:addr': b'\xe5\xbb\xb6\xe5\xae\x89', b'cf:age': b'34', b'cf:name': b'weiwei'}


In [ ]:
import happybase
import numpy as np
import pandas as pd
def get_hbase_db():
    conn = happybase.Connection('47.93.144.37')
    return conn
# conn = get_hbase_db()
# tbls = conn.tables()
# print(tbls)
# tbl_test1 = conn.table(b'test_1')
# #遍历表
# for key, data in tbl_test1.scan():
#     print(key, data)  
help(pd.DataFrame)

In [13]:
import time
print(time.asctime())
print(time.time())


Thu Jul 20 11:03:27 2017
1500519807.2679443


['CLOCK_MONOTONIC',
 'CLOCK_MONOTONIC_RAW',
 'CLOCK_PROCESS_CPUTIME_ID',
 'CLOCK_REALTIME',
 'CLOCK_THREAD_CPUTIME_ID',
 '_STRUCT_TM_ITEMS',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'altzone',
 'asctime',
 'clock',
 'clock_getres',
 'clock_gettime',
 'clock_settime',
 'ctime',
 'daylight',
 'get_clock_info',
 'gmtime',
 'localtime',
 'mktime',
 'monotonic',
 'perf_counter',
 'process_time',
 'sleep',
 'strftime',
 'strptime',
 'struct_time',
 'time',
 'timezone',
 'tzname',
 'tzset']

In [ ]:
%matplotlibimport   inline 
#要在ipy notebook里直接显示Matplotlib的输出图象，需要%inline

import matplotlib.pyplot as plt
import numpy as np
X = np.linspace(-np.pi,np.pi,256,endpoint=True)
(C,S)=np.cos(X),np.sin(X)

#这里用到了Matplotlib和numpy模块,linspace在(−π,π)之间分成共256个小段，
#并把这256个值赋予X。C,S分别是cosine和sine值（X,C,S都是numpy数组）
plt.plot(X,C)
plt.plot(X,S)

In [ ]:
# -*- coding:utf-8 -*-
import requests
url = 'http://api.map.baidu.com/staticimage/v2?ak=1XjLLEhZhQNUzd93EjU5nOGQ&mcode=666666&center=115.403874,39.914888&width=300&height=200&zoom=11'
r = requests.get(url)

if r.status_code == 200 or r.status_code == 304:
#     print(r.text)
    help(r)
else:
    print(r.status_code)


In [21]:
# aa = '123abc123'
# aa = aa.replace('123', 'nnn',1)
# print(aa)
# help(aa.replace)

import datetime
import time
# dir(time)
# help(time.strptime)
# help(datetime.datetime.now().strftime)
  
# print(time.strftime('%Y-%m-%d %H:%M:%S')) 
aa = ['23', '2', '9']
# aa.sort()
foo = lambda x, y: x<y
type(aa)
help(aa)


Help on list object:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /